# Compute GitHub Stats

## Notebook setup

In [1]:
import os
import subprocess
if os.path.exists("/var/run/secrets/kubernetes.io/serviceaccount"):
    subprocess.check_call(["pip", "install", "--user", "-r", "requirements.txt"], stderr=subprocess.STDOUT, bufsize=1)

In [2]:
# NOTE: The RuntimeWarnings (if any) are harmless. See ContinuumIO/anaconda-issues#6678.
from pandas.io import gbq
import pandas as pd
import numpy as np

from importlib import reload
import itertools

In [3]:
import getpass
import subprocess
# Configuration Variables. Modify as desired.

PROJECT = subprocess.check_output(["gcloud", "config", "get-value", "project"]).strip().decode()

In [4]:
%matplotlib

Using matplotlib backend: agg


## Setup Authorization

If you are using a service account run
%%bash

# Activate Service Account provided by Kubeflow.
gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}

If you are running using user credentials

gcloud auth application-default login

In [74]:
import datetime
month = datetime.datetime.now().month
year = datetime.datetime.now().year

num_months = 12

months = []
for i in range(num_months):
    months.append("\"{0}{1:02}\"".format(year, month))
    month -= 1
    if month == 0:
        month = 12
        year -=1

## Unique PR Creators

In [76]:
query = """
SELECT
    DATE(created_at) AS pr_date,
    actor.id,
    actor.login,
    JSON_EXTRACT(payload, '$.pull_request.user.id') as user_id,
    JSON_EXTRACT(payload, '$.pull_request.id') as pr_id,
    JSON_EXTRACT(payload, '$.pull_request.merged') as merged
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})
    AND type = 'PullRequestEvent'
    AND org.login = 'kubeflow'
    AND JSON_EXTRACT(payload, '$.action') IN ('"closed"')
""".format(",".join(months))

all_prs=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

/home/jovyan/.local/lib/python3.6/site-packages/pandas_gbq/gbq.py:555: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [77]:
# Filter PRs to merged PRs
v=all_prs["merged"].values == 'true'
merged_all_prs = all_prs.iloc[v]

In [78]:
p=pd.Series(data=merged_all_prs["user_id"].values,index=merged_all_prs["pr_date"])
p=p.sort_index()

In [111]:
# Some solutions here: https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series
# Need to figure out how to do a time based window

creators = p.rolling('28d').apply(lambda arr: pd.Series(arr).nunique())

# We need to group the days. Rolling window will create a point for each data point
creators_df = pd.DataFrame({"day": creators.index, "num_authors": creators.values})
creators_df = creators_df.groupby("day", as_index=False).max()

In [112]:
import altair as alt
chart = alt.Chart(creators_df, title= "Unique PR Authors (Last 28 Days)")
line = chart.mark_line().encode(
  x= alt.X('day', title = "Day"),
  y=alt.Y("num_authors", title="# Unique Authors"),    
)

point = line + line.mark_point()
point.interactive()

alt.LayerChart(...)

### Number Prs

In [116]:
pr_impulse=pd.Series(data=merged_all_prs["pr_id"].values,index=merged_all_prs["pr_date"])
pr_impulse=pr_impulse.sort_index()

unique_prs = pr_impulse.rolling('28d').apply(lambda arr: pd.Series(arr).nunique())

prs_df = pd.DataFrame({"day": unique_prs.index, "num_prs": unique_prs.values})
prs_df = prs_df.groupby("day", as_index=False).max()

In [117]:
chart = alt.Chart(prs_df, title= "Merged PRs (Last 28 Days)")
line = chart.mark_line().encode(
  x= alt.X('day', title = "Day"),
  y=alt.Y("num_prs", title="# PRs"),    
)

point = line + line.mark_point()
point.interactive()

alt.LayerChart(...)

## Release stats per release (quarter)

* Compute stats about a release 
* We do this based on time
* You can see a sample of the payload at [https://api.github.com/repos/kubeflow/pipelines/pulls/1038](https://api.github.com/repos/kubeflow/pipelines/pulls/1038)

In [85]:
release_months = []
year = 2019


for month in range(8, 11):
    release_months.append("\"{0}{1:02}\"".format(year, month))
        

query = """
SELECT
    DATE(created_at) AS pr_date,
    actor.id,
    actor.login,
    JSON_EXTRACT(payload, '$.pull_request.merged') as merged,
    JSON_EXTRACT(payload, '$.pull_request.id') as pr_id,
    JSON_EXTRACT(payload, '$.pull_request.url') as pr_url,
    JSON_EXTRACT(payload, '$.pull_request.user.id') as user_id
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})
    AND type = 'PullRequestEvent'
    AND org.login = 'kubeflow'
    AND JSON_EXTRACT(payload, '$.action') IN ('"closed"')
""".format(",".join(release_months))

prs=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

In [86]:
# Filter PRs to merged PRs
v=prs["merged"].values == 'true'
merged_prs = prs.iloc[v]

In [87]:
unique_pr_logins = prs["user_id"].unique()
unique_prs = prs["pr_id"].unique()

merged_unique_logins = merged_prs["user_id"].unique()
merged_unique_prs = merged_prs["pr_id"].unique()
print("Number of unique pr authors (merged & unmerged) {0}".format(unique_pr_logins.shape))
print("Number of unique prs (merged & unmerged) {0}".format(unique_prs.shape))
print("Number of unique pr authors (merged) {0}".format(merged_unique_logins.shape))
print("Number of unique prs (merged) {0}".format(merged_unique_prs.shape))

Number of unique pr authors (merged & unmerged) (190,)
Number of unique prs (merged & unmerged) (1788,)
Number of unique pr authors (merged) (155,)
Number of unique prs (merged) (1492,)


## Get a list of distinct actions

* Here's a list of events in the [api](https://developer.github.com/v4/union/pullrequesttimelineitems/)
* It looks like these are different from the ones in the github archive

In [88]:
query = """
SELECT
    distinct JSON_EXTRACT(payload, '$.action')
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})    
""".format(",".join(months))

actions=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

In [89]:
actions

,f0_
0,"""opened"""
1,"""started"""
2,None
3,"""published"""
4,"""created"""
5,"""added"""
6,"""reopened"""
7,"""closed"""


## New Issues Last 28 Days

In [90]:
query = """
SELECT
    DATE(created_at) AS issue_date,
    actor.id,
    actor.login,
    JSON_EXTRACT(payload, '$.pull_request.id') as issue_id,
    JSON_EXTRACT(payload, '$.pull_request.url') as issue_url  
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})
    AND type = 'IssuesEvent'
    AND org.login = 'kubeflow'
    AND JSON_EXTRACT(payload, '$.action') IN ('"opened"')
""".format(",".join(months))

issues=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

/home/jovyan/.local/lib/python3.6/site-packages/pandas_gbq/gbq.py:555: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [91]:
issue_counts=issues["issue_date"].value_counts()
issue_counts=issue_counts.sort_index()
rolling_issue_count = issue_counts.rolling('28d').sum()

In [92]:
import matplotlib
from matplotlib import pylab
matplotlib.rcParams.update({'font.size': 22})
hf = pylab.figure()
hf.set_size_inches(18.5, 10.5)
pylab.plot(rolling_issue_count, linewidth=5)
ha = pylab.gca()
ha.set_title("New Kubeflow Issues (28 Days)")
ha.set_xlabel("Date")
ha.set_ylabel("# Of Issues")

Text(0, 0.5, '# Of Issues')

## GetSomeSampleIssue Events

In [93]:
query = """
SELECT
    *
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})
    AND type = 'IssuesEvent'
    AND org.login = 'kubeflow'
    
    limit 20
""".format(",".join(months))

events=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

In [94]:
events

,type,public,payload,repo,actor,org,created_at,id,other
0,IssuesEvent,True,"{""action"":""opened"",""issue"":{""url"":""https://api...","{'id': 177896927, 'name': 'kubeflow/metadata',...","{'id': 1383056, 'login': 'kwasi', 'gravatar_id...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 22:16:30+00:00,10773687172,"{""actor"":{""display_login"":""kwasi""}}"
1,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 112647343, 'name': 'kubeflow/kubeflow',...","{'id': 26384082, 'login': 'stale[bot]', 'grava...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 20:18:42+00:00,10773383242,"{""actor"":{""display_login"":""stale""}}"
2,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 112647343, 'name': 'kubeflow/kubeflow',...","{'id': 26384082, 'login': 'stale[bot]', 'grava...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 21:18:45+00:00,10773540968,"{""actor"":{""display_login"":""stale""}}"
3,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 178075572, 'name': 'kubeflow/kfserving'...","{'id': 20407524, 'login': 'k8s-ci-robot', 'gra...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 02:01:43+00:00,10770687104,"{""actor"":{""display_login"":""k8s-ci-robot""}}"
4,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 112647343, 'name': 'kubeflow/kubeflow',...","{'id': 777219, 'login': 'jlewi', 'gravatar_id'...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 20:00:49+00:00,10773336386,"{""actor"":{""display_login"":""jlewi""}}"
5,IssuesEvent,True,"{""action"":""opened"",""issue"":{""url"":""https://api...","{'id': 133100880, 'name': 'kubeflow/pipelines'...","{'id': 777219, 'login': 'jlewi', 'gravatar_id'...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 19:56:58+00:00,10773325771,"{""actor"":{""display_login"":""jlewi""}}"
6,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 178075572, 'name': 'kubeflow/kfserving'...","{'id': 20407524, 'login': 'k8s-ci-robot', 'gra...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 16:19:40+00:00,10772712638,"{""actor"":{""display_login"":""k8s-ci-robot""}}"
7,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 133100880, 'name': 'kubeflow/pipelines'...","{'id': 777219, 'login': 'jlewi', 'gravatar_id'...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 20:01:48+00:00,10773339049,"{""actor"":{""display_login"":""jlewi""}}"
8,IssuesEvent,True,"{""action"":""closed"",""issue"":{""url"":""https://api...","{'id': 128353922, 'name': 'kubeflow/website', ...","{'id': 20407524, 'login': 'k8s-ci-robot', 'gra...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 02:47:41+00:00,10770792321,"{""actor"":{""display_login"":""k8s-ci-robot""}}"
9,IssuesEvent,True,"{""action"":""opened"",""issue"":{""url"":""https://api...","{'id': 133100880, 'name': 'kubeflow/pipelines'...","{'id': 777219, 'login': 'jlewi', 'gravatar_id'...","{'id': 33164907, 'login': 'kubeflow', 'gravata...",2019-11-02 20:12:01+00:00,10773366395,"{""actor"":{""display_login"":""jlewi""}}"


## Get some sample pull request events 

* Want to inspect the data


In [95]:
query = """
SELECT
    *
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})
    AND type = 'PullRequestEvent'
    AND org.login = 'kubeflow'
    
    limit 20
""".format(",".join(months))

events=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

/home/jovyan/.local/lib/python3.6/site-packages/pandas_gbq/gbq.py:555: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [96]:
import pprint
import json
data = json.loads(events["payload"].values[3])
pprint.pprint(data)

{'action': 'closed',
 'number': 1112,
 'pull_request': {'_links': {'comments': {'href': 'https://api.github.com/repos/kubeflow/website/issues/1112/comments'},
                             'commits': {'href': 'https://api.github.com/repos/kubeflow/website/pulls/1112/commits'},
                             'html': {'href': 'https://github.com/kubeflow/website/pull/1112'},
                             'issue': {'href': 'https://api.github.com/repos/kubeflow/website/issues/1112'},
                             'review_comment': {'href': 'https://api.github.com/repos/kubeflow/website/pulls/comments{/number}'},
                             'review_comments': {'href': 'https://api.github.com/repos/kubeflow/website/pulls/1112/comments'},
                             'self': {'href': 'https://api.github.com/repos/kubeflow/website/pulls/1112'},
                             'statuses': {'href': 'https://api.github.com/repos/kubeflow/website/statuses/f7a8f4d4247b84ed31988b7e9657a3355026de9a'}},
   

In [97]:
data["pull_request"]["id"]

312895771

## Get Distinct Types

In [98]:
query = """
SELECT
    distinct type
  FROM `githubarchive.month.*`
  WHERE
    _TABLE_SUFFIX IN ({0})    
    AND org.login = 'kubeflow'
    
    limit 20
""".format(",".join(months))

events=gbq.read_gbq(str(query), dialect='standard', project_id=PROJECT)

In [99]:
events

,type
0,CreateEvent
1,MemberEvent
2,IssueCommentEvent
3,ReleaseEvent
4,CommitCommentEvent
5,IssuesEvent
6,GollumEvent
7,DeleteEvent
8,PullRequestReviewCommentEvent
9,WatchEvent
